In [11]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors, PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.QED import qed
from rdkit.Chem import QED, rdMolDescriptors, FilterCatalog
from utils.docking import cal_docking
from utils.metric import *
from rdkit.Chem.rdForceFieldHelpers import UFFOptimizeMolecule
from rdkit.Chem.Descriptors import qed
from utils.sascorer import compute_sa_score
from utils.misc import *
import matplotlib.pyplot as plt
from tabulate import tabulate

In [12]:
# Load configs
config = load_config('./configs/rl.yml')
seed_all(config.sample.seed)

In [13]:
sort_metric = 'Qvina' # Qvina QED SA 

ours_path = 'eval_results/baselines_results/AMG.pt'
flag_path = 'eval_results/baselines_results/FLAG.pt'
resgen_path = 'eval_results/baselines_results/ResGen.pt'
decompdiff_path = 'eval_results/baselines_results/DecompDiff.pt'
targetdiff_path = 'eval_results/baselines_results/TargetDiff.pt'
pocket2mol_path = 'eval_results/baselines_results/Pocket2Mol.pt'

## Cal metrics

In [14]:
def cal_high_affinity(results, top_num, sort_metric, test_path='eval_results/baselines_results/testset.pt'):
    high_affinity_result = []
    mpbg_list = []
    test_results = torch.load(test_path)
    
    for test_name, test_df in test_results.items():
        for result_name, result_df in results.items():
            if test_name.split('-')[0] == result_name.split('-')[0]:
                test_value = test_df['Qvina'].mean()
                results_value = result_df.head(100) 
                 
                if sort_metric == 'Qvina':
                    results_value.sort_values(by=sort_metric, inplace=True, ascending=True)
                else:
                    results_value.sort_values(by=sort_metric, inplace=True, ascending=False)  
          
                # cal MPBG
                min_test_value = test_df['Qvina'].mean()
                sum_mpbg = 0
                lenth = 0
         
                for qvina in results_value.head(top_num)['Qvina'].values:
                    if not np.isnan(qvina):
                        sum_mpbg += ((min_test_value - qvina) / min_test_value)
                        lenth += 1

                if lenth:
                    mpbg_list.append(sum_mpbg / lenth)
                else:
                    mpbg_list.append(0)
                    
                results_value = results_value.head(top_num)
                high_affinity = (results_value['Qvina'] < test_value).sum()
                high_affinity_result.append((high_affinity / top_num))
   
    return np.mean(high_affinity_result), np.mean(mpbg_list)


def get_top_n_dfs(df, top_n_list):
    """
    Returns a dictionary of DataFrames for each specified top N.
    """
    return {n: df.head(n) for n in top_n_list}

def calculate_statistics(results, top_dfs, top_nums, sort_metric):
    """
    Calculates statistics for given top N DataFrames, adds a title, and prints them in a table format with top N as rows.
    """
    all_stats = []
    for top_num in top_nums:
        top_df = top_dfs[top_num]
        top_mean = top_df.mean().round(3)
        top_std = top_df.std().round(2)

        high_affinity_mean, mpbg_mean = cal_high_affinity(results, top_num, sort_metric)

        stats = {
            'Top': f'Top {top_num}',
            'Qvina': f"{top_mean['Qvina']} ± {top_std['Qvina']}",
            'HA': f"{high_affinity_mean.round(3)}",
            'MPBG': f"{mpbg_mean.round(2)}",
            'QED': f"{top_mean['QED']} ± {top_std['QED']}",
            'SA': f"{top_mean['SA']} ± {top_std['SA']}",
            'Lipinski': f"{top_mean['Lip']} ± {top_std['Lip']}",  
        }

        all_stats.append(stats)

    stats_df = pd.DataFrame(all_stats).set_index('Top')
    print(tabulate(stats_df, headers='keys', tablefmt='pretty', showindex=True))
    
def cal_result(path, sort_metric, show_result=False, return_df=False):
    top_n_list = [1, 3, 5, 10]
    top_dfs = {n: [] for n in top_n_list}
    
    results = torch.load(path)
 
    for name, df in results.items():
        eval_df = df.copy() 
        eval_df = eval_df.head(100)        
        if sort_metric == 'Qvina':
            eval_df.sort_values(by=sort_metric, inplace=True, ascending=True)
        else:
            eval_df.sort_values(by=sort_metric, inplace=True, ascending=False)
  
        for n, top_df in get_top_n_dfs(eval_df, top_n_list).items():
            top_dfs[n].append(top_df)
    
    top_dfs = {n: pd.concat(dfs) for n, dfs in top_dfs.items()}
    if return_df:
        return top_dfs
    if show_result:
        calculate_statistics(results, top_dfs, top_n_list, sort_metric)


## Ours

In [15]:
cal_result(ours_path, sort_metric, show_result=True)  

+--------+----------------+-------+-------+--------------+--------------+--------------+
|  Top   |     Qvina      |  HA   | MPBG  |     QED      |      SA      |   Lipinski   |
+--------+----------------+-------+-------+--------------+--------------+--------------+
| Top 1  | -10.201 ± 1.62 | 0.98  | -1.52 | 0.524 ± 0.15 | 0.514 ± 0.1  | 4.82 ± 0.39  |
| Top 3  | -9.765 ± 1.51  | 0.973 | -1.43 | 0.538 ± 0.15 | 0.514 ± 0.11 | 4.893 ± 0.31 |
| Top 5  | -9.508 ± 1.47  | 0.958 | -1.37 | 0.553 ± 0.15 | 0.512 ± 0.1  | 4.91 ± 0.29  |
| Top 10 | -9.102 ± 1.42  | 0.928 | -1.27 | 0.56 ± 0.14  | 0.515 ± 0.11 | 4.932 ± 0.25 |
+--------+----------------+-------+-------+--------------+--------------+--------------+


## FLAG

In [16]:
cal_result(flag_path, sort_metric, show_result=True) 

+--------+---------------+-------+-------+--------------+--------------+--------------+
|  Top   |     Qvina     |  HA   | MPBG  |     QED      |      SA      |   Lipinski   |
+--------+---------------+-------+-------+--------------+--------------+--------------+
| Top 1  | -9.837 ± 1.6  | 0.96  | -1.45 | 0.436 ± 0.17 | 0.397 ± 0.13 | 4.28 ± 0.94  |
| Top 3  | -9.414 ± 1.48 | 0.93  | -1.34 | 0.458 ± 0.17 | 0.406 ± 0.12 | 4.377 ± 0.83 |
| Top 5  | -9.159 ± 1.42 | 0.908 | -1.29 | 0.468 ± 0.16 | 0.406 ± 0.12 | 4.432 ± 0.81 |
| Top 10 | -8.756 ± 1.37 | 0.873 | -1.19 | 0.476 ± 0.16 | 0.415 ± 0.12 | 4.493 ± 0.76 |
+--------+---------------+-------+-------+--------------+--------------+--------------+


## ResGen

In [17]:
cal_result(resgen_path, sort_metric, show_result=True) 

+--------+---------------+-------+-------+--------------+--------------+--------------+
|  Top   |     Qvina     |  HA   | MPBG  |     QED      |      SA      |   Lipinski   |
+--------+---------------+-------+-------+--------------+--------------+--------------+
| Top 1  | -8.693 ± 2.3  | 0.91  | -0.96 | 0.567 ± 0.15 | 0.796 ± 0.12 | 4.94 ± 0.28  |
| Top 3  | -8.429 ± 2.25 | 0.883 | -0.89 | 0.566 ± 0.16 | 0.804 ± 0.12 | 4.913 ± 0.33 |
| Top 5  | -8.292 ± 2.21 | 0.856 | -0.85 | 0.571 ± 0.16 | 0.802 ± 0.11 | 4.912 ± 0.34 |
| Top 10 | -8.083 ± 2.16 | 0.824 | -0.8  | 0.579 ± 0.16 |  0.8 ± 0.11  | 4.908 ± 0.34 |
+--------+---------------+-------+-------+--------------+--------------+--------------+


## DecompDiff

In [18]:
cal_result(decompdiff_path, sort_metric, show_result=True) 

+--------+---------------+-------+-------+--------------+--------------+--------------+
|  Top   |     Qvina     |  HA   | MPBG  |     QED      |      SA      |   Lipinski   |
+--------+---------------+-------+-------+--------------+--------------+--------------+
| Top 1  | -8.485 ± 1.99 | 0.95  | -0.99 | 0.552 ± 0.19 | 0.656 ± 0.15 | 4.51 ± 0.82  |
| Top 3  | -8.116 ± 2.03 | 0.92  | -0.78 | 0.544 ± 0.2  | 0.66 ± 0.15  | 4.517 ± 0.85 |
| Top 5  | -7.946 ± 2.01 | 0.906 | -0.72 | 0.541 ± 0.2  | 0.662 ± 0.15 | 4.492 ± 0.89 |
| Top 10 | -7.715 ± 1.97 | 0.864 | -0.65 | 0.539 ± 0.21 | 0.666 ± 0.15 | 4.435 ± 0.96 |
+--------+---------------+-------+-------+--------------+--------------+--------------+


## TargetDiff

In [19]:
cal_result(targetdiff_path, sort_metric, show_result=True) 

+--------+---------------+-------+-------+--------------+--------------+--------------+
|  Top   |     Qvina     |  HA   | MPBG  |     QED      |      SA      |   Lipinski   |
+--------+---------------+-------+-------+--------------+--------------+--------------+
| Top 1  | -9.278 ± 1.75 | 0.94  | -1.22 | 0.426 ± 0.2  | 0.476 ± 0.12 | 4.42 ± 0.82  |
| Top 3  | -8.892 ± 1.68 | 0.923 | -1.12 | 0.457 ± 0.21 | 0.502 ± 0.12 | 4.453 ± 0.88 |
| Top 5  | -8.686 ± 1.65 | 0.908 | -1.07 | 0.453 ± 0.21 | 0.511 ± 0.12 | 4.446 ± 0.87 |
| Top 10 | -8.37 ± 1.61  | 0.885 | -1.0  | 0.466 ± 0.2  | 0.523 ± 0.12 | 4.519 ± 0.79 |
+--------+---------------+-------+-------+--------------+--------------+--------------+


## Pocket2Mol

In [20]:
cal_result(pocket2mol_path, sort_metric, show_result=True) 

+--------+---------------+-------+-------+--------------+--------------+--------------+
|  Top   |     Qvina     |  HA   | MPBG  |     QED      |      SA      |   Lipinski   |
+--------+---------------+-------+-------+--------------+--------------+--------------+
| Top 1  | -8.785 ± 2.88 | 0.88  | -0.95 | 0.517 ± 0.14 | 0.778 ± 0.13 | 4.95 ± 0.22  |
| Top 3  | -8.525 ± 2.9  | 0.827 | -0.86 | 0.525 ± 0.15 | 0.777 ± 0.12 | 4.947 ± 0.24 |
| Top 5  | -8.395 ± 2.88 | 0.804 | -0.81 | 0.528 ± 0.15 | 0.781 ± 0.12 | 4.946 ± 0.24 |
| Top 10 | -8.19 ± 2.83  | 0.771 | -0.76 | 0.53 ± 0.15  | 0.78 ± 0.12  | 4.941 ± 0.28 |
+--------+---------------+-------+-------+--------------+--------------+--------------+
